<a href="https://colab.research.google.com/github/ambgeo/geoquantificacao/blob/main/03_S%C3%A9ries__Temporais_An%C3%A1lises_Estat%C3%ADsticas_Visualiza%C3%A7%C3%A3o_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Séries Temporais no GEE: CHIRPS (Precipitação) + MODIS LST (Temperatura Superficial)

**Objetivo:** Autenticar no GEE, definir ROI desenhando no `geemap`, extrair séries de 2024 para **CHIRPS (precipitação diária)** e **MODIS/061/MOD11A2 (temperatura superficial do dia, 8 dias)**, calcular estatísticas, plotar gráficos e mapear **precipitação acumulada** e **temperatura média**.

**Passos:**
* 1) Autenticação  
* 2) Seleção da área de estudo `roi = Map.user_roi`  
* 3) Dados de 2024 (jan–dez)  
* 4) Série temporal (colunas: `data`, `ano`, `mes`, `precip_media_mm`, `temp_acum_c_dia`)  
* 5) Gráfico da série histórica (precipitação e temperatura)  
* 6) Gráfico mensal (precipitação acumulada e temperatura média)  
* 7) Mapas: Precipitação acumulada e Temperatura média  


## 1) Autenticação

In [ ]:
import ee
try:
    ee.Authenticate()
except Exception as e:
    print('Se já autenticou anteriormente nesta máquina, pode ignorar este aviso:', e)
ee.Initialize(project='ee-scriptsambgeo')
print('GEE inicializado!')

## 2) Seleção da ROI desenhando no `geemap`
1. Execute a célula do mapa.
2. Clique no ícone **Draw** (lápis) e desenhe um **polígono**.
3. Execute a célula de captura de ROI.

In [ ]:
# Se necessário: !pip -q install geemap folium ipyleaflet
import geemap
Map = geemap.Map(center=[-15, -55], zoom=4)
Map.addLayerControl()
Map

In [ ]:
# Capturar a ROI desenhada
roi = Map.user_roi
Map.centerObject(roi, 8)
Map.addLayer(roi, {"color":"red"}, 'ROI')
print('ROI definida!')
Map

## 3) Intervalo de datas (2024) e coleções CHIRPS + MODIS LST

In [ ]:
import datetime as dt

start_py = '2024-01-01'
end_py   = '2025-01-01'

# CHIRPS DAILY (mm/dia) - banda 'precipitation'
chirps = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
          .filterBounds(roi)
          .filterDate(start_py, end_py)
          .select('precipitation'))

# MODIS Terra LST 8-day (K * 0.02 -> °C) - banda 'LST_Day_1km'
modis = (ee.ImageCollection('MODIS/061/MOD11A2')
         .filterBounds(roi)
         .filterDate(start_py, end_py)
         .select('LST_Day_1km'))

def to_celsius(img):
    lst_c = img.multiply(0.02).subtract(273.15).rename('LST_C')
    return lst_c.copyProperties(img, img.propertyNames())

modis_c = modis.map(to_celsius)

print('CHIRPS imgs:', chirps.size().getInfo(), '| MODIS imgs:', modis_c.size().getInfo())

## 4) Série temporal (ReduceRegion por imagem) → DataFrame
Resultado: colunas `data`, `ano`, `mes`, `precip_media_mm`, `temp_acum_c_dia`.

- **precip_media_mm**: média da precipitação diária (mm) na ROI (CHIRPS).  
- **temp_acum_c_dia**: temperatura **acumulada** por composição MODIS (≈ 8 dias), calculada como `média(°C) * 8` → unidade **°C·dia**.

In [ ]:
import pandas as pd

def reduce_image_mean(img, band_name, geom):
    stat = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=500 if band_name=='LST_C' else 5566,
        maxPixels=1e13
    )
    return ee.Feature(None, {
        'date': ee.Date(img.get('system:time_start')).format('YYYY-MM-dd'),
        'year': ee.Number.parse(ee.Date(img.get('system:time_start')).format('YYYY')),
        'month': ee.Number.parse(ee.Date(img.get('system:time_start')).format('MM')),
        band_name: stat.get(band_name)
    })

In [ ]:
# CHIRPS: média diária mm
chirps_fc = chirps.map(lambda im: reduce_image_mean(im, 'precipitation', roi))
chirps_df = pd.DataFrame([f['properties'] for f in chirps_fc.getInfo()['features']])
chirps_df.rename(columns={'precipitation':'precip_media_mm', 'date':'data','year':'ano','month':'mes'}, inplace=True)
chirps_df['data'] = pd.to_datetime(chirps_df['data'])

# MODIS: média °C e 'acumulado' (°C·dia) ≈ mean*8
modis_fc = modis_c.map(lambda im: reduce_image_mean(im, 'LST_C', roi))
modis_df = pd.DataFrame([f['properties'] for f in modis_fc.getInfo()['features']])
modis_df.rename(columns={'LST_C':'temp_media_c','date':'data','year':'ano','month':'mes'}, inplace=True)
modis_df['data'] = pd.to_datetime(modis_df['data'])


# Unir séries por data
df = pd.merge(chirps_df[['data','ano','mes','precip_media_mm']],
              modis_df[['data','temp_media_c']],
              on='data', how='outer').sort_values('data').reset_index(drop=True)

df.head()

## 5) Gráfico da série histórica (2024)
- Linha: **temperatura média (°C)**
- Barras: **precipitação diária média (mm)**

> Um gráfico por figura, usando apenas matplotlib.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Remove o grid padrão do seaborn
sns.set_theme(style="white")

fig, ax1 = plt.subplots(figsize=(10, 4))

# Barras = precipitação diária (azul)
ax1.bar(
    df['data'],
    df['precip_media_mm'].fillna(0),
    width=2, align='center',
    color='blue', edgecolor='none'
)
ax1.set_ylabel('Precipitação (mm/dia)')
ax1.grid(False)

# Linha = temperatura média (vermelho)
ax2 = ax1.twinx()
sns.lineplot(
    data=df, x='data', y='temp_media_c',
    marker='o', linewidth=1, color='red', ax=ax2
)
ax2.set_ylabel('Temperatura média (°C)')
ax2.grid(False)

# Eixo de datas
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
for label in ax1.get_xticklabels():
    label.set_rotation(45)
    label.set_ha('right')

plt.title('Série histórica — Precipitação (CHIRPS) e Temperatura (MODIS LST) — 2024')
fig.tight_layout()
plt.show()


## 6) Gráfico mensal (precipitação acumulada e temperatura média)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# --- 1) Preparação dos dados (filtra 2024 e cria agregações mensais) ---
d = df.copy()
d['data'] = pd.to_datetime(d['data']).dt.tz_localize(None)                 # garante datetime sem timezone
d = d[(d['data'] >= '2024-01-01') & (d['data'] < '2025-01-01')]            # mantém só 2024

# Resample mensal (início do mês) -> 12 linhas
mon = (d.set_index('data')
         .resample('MS')
         .agg(precip_mensal=('precip_media_mm','sum'),                     # mm/mês (acumulado)
              temp_mensal=('temp_media_c','mean'))                         # °C (média do mês)
         .reset_index())

# Garante exatamente 12 meses, mesmo se faltarem dados
idx = pd.date_range('2024-01-01','2024-12-01', freq='MS')
mon = (mon.set_index('data')
          .reindex(idx)
          .reset_index()
          .rename(columns={'index':'data'}))

# --- 2) Aparência geral ---
sns.set_theme(style="white")  # fundo branco; sem grid do seaborn

# --- 3) Figura e eixos ---
fig, ax1 = plt.subplots(figsize=(10, 4))  # ax1 = precipitação (barras)

# --- 4) Barras: precipitação acumulada mensal (12 valores) ---
ax1.bar(
    mon['data'],                          # 12 datas (início de cada mês)
    mon['precip_mensal'].fillna(0),       # acumulado do mês (mm)
    width=25, align='center',             # largura ~25 dias deixa visual agradável
    color='blue', edgecolor='none'
)
ax1.set_ylabel('Precipitação acumulada (mm)')
ax1.grid(False)

# --- 5) Linha: temperatura média mensal (12 valores) ---
ax2 = ax1.twinx()                         # eixo secundário para temperatura
sns.lineplot(
    data=mon, x='data', y='temp_mensal',
    marker='o', linewidth=1, color='red', ax=ax2,
    legend=False                          # evita legenda automática do seaborn
)
ax2.set_ylabel('Temperatura média (°C)')
ax2.grid(False)

# --- 6) Eixo de datas (marcas mensais e rótulos Mês/Ano) ---
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
for label in ax1.get_xticklabels():
    label.set_rotation(45)
    label.set_ha('right')

# --- 7) Legenda única, no topo central ---
handles = [
    Patch(facecolor='blue', edgecolor='none', label='Precipitação (mm/mês)'),
    Line2D([0], [0], color='red', marker='o', linewidth=1, label='Temperatura média (°C)')
]
fig.legend(handles=handles, loc='upper center', ncol=2, frameon=False, bbox_to_anchor=(0.5, 0.02))

# --- 8) Título e layout ---
plt.title('Mensal — Precipitação acumulada (CHIRPS) e Temperatura média (MODIS) — 2024', y=1.06)
fig.tight_layout(rect=[0, 0, 1, 0.92])    # reserva espaço para a legenda no topo
plt.show()


## 7) Mapas: Precipitação acumulada (mm) e Temperatura média (°C) — 2024

In [ ]:
precip_acc = chirps.sum().rename('precip_2024_mm').clip(roi)
temp_mean_img = modis_c.mean().rename('lst_mean_c_2024').clip(roi)


palette_blues = [
    '#f7fbff', '#deebf7', '#c6dbef', '#9ecae1',
    '#6baed6', '#4292c6', '#2171b5', '#08519c', '#08306b'
]

palette_warm = [
    '#ffffcc', '#ffeda0', '#fed976', '#feb24c',
    '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026'
]


# --- 3) Visualização no mapa com paletas dedicadas ---
# Precipitação acumulada (mm) — tons de azul
Map.addLayer(
    precip_acc,
    {"min": 1200, "max": 2300, "palette": palette_blues},
    "Precipitação acumulada 2024 (mm)"
)

# Temperatura média (°C) — tons quentes
Map.addLayer(
    temp_mean_img,
    {"min": 20, "max": 30, "palette": palette_warm},
    "Temperatura média 2024 (°C)"
)

Map.addLayer(roi, {"color":"red"}, 'ROI')
Map